In [1]:
## Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from ast import literal_eval

In [2]:
DATASET_NAME = 'Walmart-Amazon'

In [3]:
class LSH :
  
    # Random limits, const
    __max_rand =  1000
    __min_rand = -1000
    
    # Constructor
    def __init__(self, k, L, embedding_size = 150):
        
        # Number of hash function
        self.k = k
        
        # Number of attempts
        self.L = L
        
        # Embedding length 
        self.embedding_size = embedding_size
        
        # Random matrices
        self.normalized_random_matrices = []
        
        for i in range(self.L):
            random_matrix = np.random.randint(self.__min_rand, self.__max_rand,(self.k, self.embedding_size));
            
            # Append normalized random matrices
            self.normalized_random_matrices.append(normalize(random_matrix, axis=1, norm='l1'))
        
    
    # Locality Sensitive hash function
    def locality_sensitive_hash(self, embedding, matrix_index):
        out = 0
      
        for h in self.normalized_random_matrices[matrix_index]:
            if (np.dot(h, embedding) >= 0):
                out = (out << 1) | 1
            else:
                out = (out << 1) | 0

        return out
      
    # Divide in buckets using L-th matrix
    def divide_in_buckets(self, embeddings, matrix_index):
        out = {}
        for embedding in embeddings:
            hash = self.locality_sensitive_hash(embedding, matrix_index)
            if (hash in out):
                out[hash].append(embedding)
            else:
                out[hash] = [embedding]
            
        return out  


In [4]:
test = LSH(k=5, L=2, embedding_size=10)
print(test.normalized_random_matrices)

print(test.locality_sensitive_hash([1,2,3,4,5,6,7,8,9,10], 1))

[array([[-0.00199417,  0.09449302,  0.12778033,  0.06596104,  0.13299586,
         0.14281331, -0.00199417,  0.15201718,  0.15247737,  0.12747354],
       [-0.06120933, -0.1063735 ,  0.13088694, -0.12910415, -0.12925271,
         0.04234141,  0.10563066,  0.10087654, -0.14128658, -0.05303818],
       [-0.06561447,  0.17148431,  0.07412662,  0.11704203,  0.15144529,
         0.03085653,  0.04096471, -0.0847668 ,  0.15907076,  0.10462848],
       [-0.13265993, -0.071156  , -0.08462402,  0.11380471,  0.11694725,
        -0.17126824,  0.10347924,  0.05858586, -0.0352413 , -0.11223345],
       [-0.06069827,  0.14702114,  0.00432415, -0.11611147, -0.00176169,
        -0.08920564, -0.14397822, -0.12668161,  0.1558296 ,  0.15438821]]), array([[ 0.05328874, -0.0787068 ,  0.07647715,  0.06911929,  0.11393534,
         0.18037904,  0.11148272, -0.07848384,  0.07759197,  0.16053512],
       [ 0.06924795, -0.06552494, -0.18912882,  0.10374783,  0.14048151,
        -0.24298833,  0.04889551, -0.02258

In [5]:
## almost equals
embeddings = [[1.2345,2,3,4,5,6,10.4,8,9,10],[1,2,3,4,5,6,7,8,9,10],[1,2,3,5,5,6,7,8,9,10]]
print(len(test.divide_in_buckets(embeddings, 1)))

## not equals
embeddings = np.random.randint(-10000, 10000,(10000, 10))
print(len(test.divide_in_buckets(embeddings, 1)))


1
32


In [6]:
## TEST BLOCKING PERFORMANCE
#    Basta che per ogni tupla vado a prendere la sua corrispondente, ne calcolo
#     i vari L hash e controllo che almeno uno sia uguale e incremento un
#     contatore. La precisione è contatore/numero di tuple controllate, giusto?
def performance_test(filtered_dataset, k, L, embedding_size):
    
    match_found = 0
    
    lsh = LSH(k, L, embedding_size)
    
    # for each elemt in dataset
    for index, row in filtered_dataset.iterrows():
        x_embedding = np.array(literal_eval(row['left_table']))
        y_embedding = np.array(literal_eval(row['right_table']))
          
        x_hashs = set()
        y_hashs = set()
        for i in range(L):
            x_hashs.add(lsh.locality_sensitive_hash(x_embedding, i))
            y_hashs.add(lsh.locality_sensitive_hash(y_embedding, i))
        
        if (len(set.intersection(x_hashs, y_hashs)) > 0):
            match_found += 1
  
    
    return match_found / len(filtered_dataset.index)

In [7]:
## Open dataset 
df = pd.read_csv('../../lsh-test-data/' + DATASET_NAME + '-embeddings.csv')

## Remove 0 labled
df = df[df.label == 1]

precision_max = 0
k_max = 0
L_max = 0
for k in range(30):
    for L in range(10):
        precision = performance_test(df, k + 1, L + 1, 150)
        print("K: {0}, L: {1}, Precision:{2}".format(k + 1, L + 1, precision))
        if (precision >= precision_max):
            precision_max = precision
            k_max = k + 1
            L_max = L + 1

print("Max precision: {0}, k: {1}, L: {2}".format(precision_max, k_max, L_max))

K: 1, L: 1, Precision:0.7253886010362695
K: 1, L: 2, Precision:0.9948186528497409
K: 1, L: 3, Precision:1.0
K: 1, L: 4, Precision:0.9948186528497409
K: 1, L: 5, Precision:1.0
K: 1, L: 6, Precision:1.0
K: 1, L: 7, Precision:1.0
K: 1, L: 8, Precision:1.0
K: 1, L: 9, Precision:1.0
K: 1, L: 10, Precision:1.0
K: 2, L: 1, Precision:0.5854922279792746
K: 2, L: 2, Precision:0.8549222797927462
K: 2, L: 3, Precision:0.9844559585492227
K: 2, L: 4, Precision:1.0
K: 2, L: 5, Precision:1.0
K: 2, L: 6, Precision:1.0
K: 2, L: 7, Precision:1.0
K: 2, L: 8, Precision:1.0
K: 2, L: 9, Precision:1.0
K: 2, L: 10, Precision:1.0
K: 3, L: 1, Precision:0.48704663212435234
K: 3, L: 2, Precision:0.772020725388601
K: 3, L: 3, Precision:0.9533678756476683
K: 3, L: 4, Precision:0.9637305699481865
K: 3, L: 5, Precision:0.9948186528497409
K: 3, L: 6, Precision:1.0
K: 3, L: 7, Precision:1.0
K: 3, L: 8, Precision:1.0
K: 3, L: 9, Precision:1.0
K: 3, L: 10, Precision:1.0
K: 4, L: 1, Precision:0.3316062176165803
K: 4, L: 2,

K: 21, L: 4, Precision:0.09844559585492228
K: 21, L: 5, Precision:0.09844559585492228
K: 21, L: 6, Precision:0.12953367875647667
K: 21, L: 7, Precision:0.13471502590673576
K: 21, L: 8, Precision:0.15025906735751296
K: 21, L: 9, Precision:0.18134715025906736
K: 21, L: 10, Precision:0.21243523316062177
K: 22, L: 1, Precision:0.0051813471502590676
K: 22, L: 2, Precision:0.05181347150259067
K: 22, L: 3, Precision:0.08808290155440414
K: 22, L: 4, Precision:0.12953367875647667
K: 22, L: 5, Precision:0.11917098445595854
K: 22, L: 6, Precision:0.09326424870466321
K: 22, L: 7, Precision:0.12953367875647667
K: 22, L: 8, Precision:0.13989637305699482
K: 22, L: 9, Precision:0.19170984455958548
K: 22, L: 10, Precision:0.13989637305699482
K: 23, L: 1, Precision:0.031088082901554404
K: 23, L: 2, Precision:0.025906735751295335
K: 23, L: 3, Precision:0.07772020725388601
K: 23, L: 4, Precision:0.06217616580310881
K: 23, L: 5, Precision:0.06217616580310881
K: 23, L: 6, Precision:0.08808290155440414
K: 23